## XGboost regressor

#### More Grid searches and continual parameter tuning

In [1]:
import os
import pandas as pd
import numpy as np


from xgboost import XGBRegressor

#### Read in merged data

In [7]:
filepath = '../../processed_data/all_processed.csv'
enviro = pd.read_csv(filepath)

In [8]:
enviro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25874 entries, 0 to 25873
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zip                          25874 non-null  int64  
 1   total population             25874 non-null  int64  
 2   ozone                        25671 non-null  float64
 3   ozone pctl                   25671 non-null  float64
 4   pm2.5                        25628 non-null  float64
 5   pm2.5 pctl                   25628 non-null  float64
 6   diesel pm                    25874 non-null  float64
 7   diesel pm pctl               25874 non-null  float64
 8   pesticides                   25874 non-null  float64
 9   pesticides pctl              25874 non-null  float64
 10  traffic                      25783 non-null  float64
 11  traffic pctl                 25783 non-null  float64
 12  cleanup sites                25874 non-null  float64
 13  cleanup sites pc

## Third XGBoost Regression model with different Grid search parameters.

In [6]:
X = all_enviro[['total population',
 'ozone',
 'pm2.5',
 'diesel pm',
 'pesticides',
 'traffic',
 'cleanup sites',
 'groundwater threats',
 'haz. waste',
 'imp. water bodies',
 'solid waste',
 'pollution burden',
 'low birth weight',
 'education',
 'linguistic isolation',
 'poverty',
 'pop. char. ',
 'drinking water',
 'tox. release',
 'unemployment',
 'ces_per',
 'cardiovascular disease',
 'housing burden',
 'est total',
 'est gen',
 'est cold',
 'est farm',
 'est other']]
y = all_enviro['asthma']

KeyError: "['est total', 'est gen', 'est cold', 'est farm', 'est other'] not in index"